# Question 15.2.1
 
In the videos, we saw the “diet problem”. (The diet problem is one of the first large-scale optimization problems to be studied in practice. Back in the 1930’s and 40’s, the Army wanted to meet the nutritional requirements of its soldiers while minimizing the cost.) In this homework you get to solve a diet problem with real data. The data is given in the file diet.xls.  
 
1. Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints, and solve it using PuLP.  Turn in your code and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs, oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!) 

In [1]:
import pandas as pd
import logging
from pulp import *

In [2]:
#Create input datasets for the homework
diet = pd.read_excel('./diet.xls')
diet_large = pd.read_excel('./diet_large.xls',header=1)

In [3]:
#Based on the office hours review we can see that the minimum and maximum nutritional constraints was not a part of the question, but
#but it's the last two rows at the end of the dataframe that contain the data for these constraints
max_constraints = diet.iloc[len(diet)-1][4:]
print(max_constraints)
print('\n')
min_constraints = diet.iloc[len(diet)-2][4:]
print(min_constraints)

diet = diet.iloc[0:64]

Cholesterol mg       240
Total_Fat g           70
Sodium mg           2000
Carbohydrates g      450
Dietary_Fiber g      250
Protein g            100
Vit_A IU           10000
Vit_C IU            5000
Calcium mg          1500
Iron mg               40
Name: 66, dtype: object


Cholesterol mg       30
Total_Fat g          20
Sodium mg           800
Carbohydrates g     130
Dietary_Fiber g     125
Protein g            60
Vit_A IU           1000
Vit_C IU            400
Calcium mg          700
Iron mg              10
Name: 65, dtype: object


In [4]:
from pandas import DataFrame
def create_input_dictionaries(df: DataFrame, fooditems: list):
    try:
        #Create parent dict variable which will contain all of the nutrient dictionaries
        parent_dict = {}
        
        #Take all of the nutrients columns and filter dataframe to only include nutrient columns
        nutrients = df.columns[4:]
        nutrients_df = df[nutrients]
        
        #Iterate through the columns and zip the contents of all food and nutritional elements
        for col in nutrients_df.columns:
            parent_dict[col] = dict(zip(fooditems,df[col]))
        return parent_dict
    except Exception as e:
        raise(e)
    
# Create a list of the food items
food_items = list(diet['Foods'])

nutritional_dict = create_input_dictionaries(diet,food_items)

In [5]:
#Create variables required for the model
food_vars = LpVariable.dicts("Food",food_items,lowBound=0,cat='Continuous')


# Create a dictinary of costs for all food items
costs = dict(zip(food_items,diet['Price/ Serving']))

In [6]:
prob = LpProblem("Military Diet Problem",LpMinimize)

#Create the objective function which is meant to contain the 
prob += lpSum([costs[i]*food_vars[i] for i in food_items])

#Loops through and create the linear program constraints
def create_constraints(linear_model: object, nutr_dict: dict):
    try:
        for nutrient, nutrient_values in nutr_dict.items():
            linear_model += lpSum([nutrient_values[f] * food_vars[f] for f in food_items]) >= min_constraints[nutrient]
            linear_model += lpSum([nutrient_values[f] * food_vars[f] for f in food_items]) <= max_constraints[nutrient]
            
        return linear_model
            
    except Exception as e:
        raise(e)
        
        
prob_final = create_constraints(prob, nutritional_dict)

c:\users\mjpearl\desktop\omsa\isye-6501-oan\hw12\env_hw12\lib\site-packages\pulp\pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [7]:
prob_final.solve()

1

In [8]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob_final.status])

Status: Optimal


In [9]:
for v in prob_final.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Food_Celery,_Raw = 52.64371
Food_Frozen_Broccoli = 0.25960653
Food_Lettuce,Iceberg,Raw = 63.988506
Food_Oranges = 2.2929389
Food_Poached_Eggs = 0.14184397
Food_Popcorn,Air_Popped = 13.869322


As you can see, the variables we see in the output are aligned with the expected output. Therefore, our solver is working correctly.

# Question 15.2.2 
2. Please add to your model the following constraints (which might require adding more variables) and solve the new model: 
    
    a. If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will need two variables for each food i: whether it is chosen, and how much is part of the diet. You’ll also need to write a constraint to link them.)

In [10]:
#a) Additional variables to be created as part of problem 2
#Create variables required for the model
food_chosen = LpVariable.dicts("Food_Chosen",food_items,0,1,cat='Binary')
#Create constraint for the new model

for i in food_items:
    # Sets minimum to 0.1
    prob_final += food_vars[i] >= 0.1*food_chosen[i]
    
    # Ties food value with binary value
    prob_final += food_vars[i] <= 9001*food_chosen[i]


b.) Frozen Broccoli and Raw Celery are mutually exclusive

In [11]:
prob_final += food_chosen['Frozen Broccoli'] + food_chosen['Celery, Raw'] == 1


c.) At least 3 proteins required which don't seem ambiguous

In [12]:
# List of necessary proteins containing either meat/poultry/fish/eggs
proteins = [
       'Bologna,Turkey', 'Frankfurter, Beef','Ham,Sliced,Extralean',
       'Hamburger W/Toppings', 'Hotdog, Plain', 'Kielbasa,Prk',
       'Pizza W/Pepperoni', 'Poached Eggs',
       'Pork', 'Roasted Chicken', 'Sardines in Oil',
       'Scrambled Eggs',
       'Splt Pea&Hamsoup', 'Vegetbeef Soup',
       'White Tuna in Water']

In [13]:
# Build constraint for protein
prob_final += lpSum([food_chosen[p] for p in proteins]) >= 3

In [14]:
prob_final.solve()

1

In [15]:
for v in prob_final.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Food_Celery,_Raw = 38.632607
Food_Chosen_Celery,_Raw = 1.0
Food_Chosen_Kielbasa,Prk = 1.0
Food_Chosen_Lettuce,Iceberg,Raw = 1.0
Food_Chosen_Oranges = 1.0
Food_Chosen_Peanut_Butter = 1.0
Food_Chosen_Poached_Eggs = 1.0
Food_Chosen_Popcorn,Air_Popped = 1.0
Food_Chosen_Scrambled_Eggs = 1.0
Food_Kielbasa,Prk = 0.1
Food_Lettuce,Iceberg,Raw = 86.960747
Food_Oranges = 3.1807762
Food_Peanut_Butter = 2.7388077
Food_Poached_Eggs = 0.1
Food_Popcorn,Air_Popped = 13.083035
Food_Scrambled_Eggs = 0.1


Therefore, we've calculated the optimal solution for adding the additional constraints to the existing model. We now have more variables introduced as part of the constraints being added.

# Optional Part
If you want to see what a more full-sized problem would look like, try solving your models for the file diet_large.xls, which is a low-cholesterol diet model (rather than minimizing cost, the goal is to minimize cholesterol intake).  I don’t know anyone who’d want to eat this diet – the optimal solution includes dried chrysanthemum garland, raw beluga whale flipper, freeze-dried parsley, etc. – which shows why it’s necessary to add additional constraints beyond the basic ones we saw in the video!  [Note: there are many optimal solutions, all with zero cholesterol, so you might get a different one.  It probably won’t be much more appetizing than mine.]

In [16]:
#Build min and max constraints df for the diet_large dataset
max_constraints_large = diet_large.iloc[len(diet_large)-1][2:]
print(max_constraints_large)
print('\n')
min_constraints_large = diet_large.iloc[len(diet_large)-3][2:]
print(min_constraints_large)

diet_large_v2 = diet_large.iloc[0:len(diet_large)-4]

Carbohydrate, by difference       1000000
Energy                            1000000
Water                             1000000
Energy.1                            1e+06
Calcium, Ca                          2500
Iron, Fe                               45
Magnesium, Mg                         400
Phosphorus, P                        4000
Potassium, K                      1000000
Sodium, Na                           2300
Zinc, Zn                               40
Copper, Cu                             10
Manganese, Mn                          11
Selenium, Se                          400
Vitamin A, RAE                       3000
Vitamin E (alpha-tocopherol)         1000
Vitamin D                            2000
Vitamin C, total ascorbic acid       2000
Thiamin                           1000000
Riboflavin                        1000000
Niacin                                 35
Pantothenic acid                  1000000
Vitamin B-6                           100
Folate, total                     

In [18]:
#Now apply the constraints required for the large dataset
prob_large = LpProblem("Military Diet Problem Large Dataset",LpMinimize)
diet_large_v2

# Create a list of the food items and dictionary 
food_items_large = list(diet_large_v2['Long_Desc'])
nutritional_dict_large = create_input_dictionaries(diet_large_v2,food_items_large)

#Create variables required for the model
food_vars_large = LpVariable.dicts("Food",food_items_large,lowBound=0,cat='Continuous')

# Create a dictionary for cholesterol to minimize
cholesterol = dict(zip(food_items_large,diet_large_v2['Cholesterol']))

#Create the objective function which is meant to contain the 
prob_large += lpSum([cholesterol[i]*food_vars_large[i] for i in food_items_large])

#prob_large
#nutritional_dict_large.keys()
#food_items_large
#prob_large_final = create_constraints(prob_large, nutritional_dict_large)